In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

C:\Users\jap090020\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [15]:
nsim = 10000
tvec = [10,25,50,100]
svec = [0.5,1,2]
rvec = [-0.75,-0.5,-0.25,0,0.25,0.5,0.75]

outp = np.zeros((len(svec)*len(rvec)*len(tvec),nsim,5))
rowcnt = 0
for sig in svec:
    for rho in rvec:
        for T in tvec:
            print(sig,rho,T)
            for isim in range(nsim):
                y = np.random.normal(0,sig,size=(T+100,1))
                for t in range(1,T+100):
                    y[t] += rho*y[t-1]
                y = y[100:]
                y_0 = y[2:T]
                y_1 = y[1:T-1]
                xdata = sm.add_constant(y_1)
                outp[rowcnt,isim,0] = sig
                outp[rowcnt,isim,1] = rho
                outp[rowcnt,isim,2] = T
                outp[rowcnt,isim,3] = smf.OLS(y_0,y_1).fit().params
                outp[rowcnt,isim,4] = smf.OLS(y_0,xdata).fit().params[1]
            rowcnt += 1
#0.5657013738527894*0.75**2+0.25
outp = np.hstack((outp.mean(1),(outp**2).var(1)[:,3:]))
outp = pd.DataFrame(outp)
columns = ('sig','rho','sample','E_b_nocon','E_b_con','V_b_nocon','V_b_con')
outp.columns = columns

0.5 -0.75 10
0.5 -0.75 25
0.5 -0.75 50
0.5 -0.75 100
0.5 -0.5 10
0.5 -0.5 25
0.5 -0.5 50
0.5 -0.5 100
0.5 -0.25 10
0.5 -0.25 25
0.5 -0.25 50
0.5 -0.25 100
0.5 0 10
0.5 0 25
0.5 0 50
0.5 0 100
0.5 0.25 10
0.5 0.25 25
0.5 0.25 50
0.5 0.25 100
0.5 0.5 10
0.5 0.5 25
0.5 0.5 50
0.5 0.5 100
0.5 0.75 10
0.5 0.75 25
0.5 0.75 50
0.5 0.75 100
1 -0.75 10
1 -0.75 25
1 -0.75 50
1 -0.75 100
1 -0.5 10
1 -0.5 25
1 -0.5 50
1 -0.5 100
1 -0.25 10
1 -0.25 25
1 -0.25 50
1 -0.25 100
1 0 10
1 0 25
1 0 50
1 0 100
1 0.25 10
1 0.25 25
1 0.25 50
1 0.25 100
1 0.5 10
1 0.5 25
1 0.5 50
1 0.5 100
1 0.75 10
1 0.75 25
1 0.75 50
1 0.75 100
2 -0.75 10
2 -0.75 25
2 -0.75 50
2 -0.75 100
2 -0.5 10
2 -0.5 25
2 -0.5 50
2 -0.5 100
2 -0.25 10
2 -0.25 25
2 -0.25 50
2 -0.25 100
2 0 10
2 0 25
2 0 50
2 0 100
2 0.25 10
2 0.25 25
2 0.25 50
2 0.25 100
2 0.5 10
2 0.5 25
2 0.5 50
2 0.5 100
2 0.75 10
2 0.75 25
2 0.75 50
2 0.75 100


In [17]:
model1 = smf.ols('I(E_b_nocon-rho)~I(rho/sample)',data=outp).fit()
model2 = smf.ols('I(E_b_con-rho)~I((1+4*rho+rho**2)/sample)',data=outp).fit()
model3 = smf.ols('I(E_b_con-rho)~I((1+rho)/sample)',data=outp).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     I(E_b_nocon - rho)   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                 1.083e+04
Date:                Wed, 27 Jun 2018   Prob (F-statistic):           7.29e-89
Time:                        17:53:14   Log-Likelihood:                 345.62
No. Observations:                  84   AIC:                            -687.2
Df Residuals:                      82   BIC:                            -682.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.0002      0.000      0.540      0.591      -0.001       0.001
I(rho / sample)    -1.6514      0.016   -104.053      0.000      -1.683      -1.620
==============================================================================
Omnibus:                        0.266   Durbin-Watson:                   1.552
Prob(Omnibus):                  0.876   Jarque-Bera (JB):                0.238
Skew:                          -0.123   Prob(JB):                        0.888
Kurtosis:                       2.911   Cond. No.                         36.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""